# Basic Example: Prompt+Model+OutputParser

- Author: [ChangJun Lee](https://www.linkedin.com/in/cjleeno1/)
- Peer Review: [Erika Park](https://www.linkedin.com/in/yeonseo-park-094193198/), [Wooseok Jeong](https://github.com/jeong-wooseok)
- Proofread : [Q0211](https://github.com/Q0211)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/06-LangChain-Expression-Language(LCEL).ipynb)
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/06-LangChain-Expression-Language(LCEL).ipynb)


## Overview

The most fundamental and commonly used case involves linking a prompt template with a model. To illustrate how this works, let us create a chain that asks for the capital cities of various countries.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Utilizing Prompt Templates](#utilizing-prompt-templates)
- [Chain Creation](#chain-creation)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it. 

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [3]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv(override=True)

True

In [4]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

True

## Utilizing Prompt Templates

```PromptTemplate```

- A prompt template is used to create a complete prompt string by incorporating the user's input variables.
- Usage
  - ```template```: A template string is a predefined format where curly braces '{}' are used to represent variables.

  - ```input_variables```: The names of the variables to be inserted within the curly braces are defined as a list.

```input_variables```

- ```input_variables``` is a list that defines the names of the variables used in the ```PromptTemplate```.

In [5]:
from langchain_core.prompts import PromptTemplate

The ```from_template()``` method is used to create a ```PromptTemplate``` object.

In [8]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [9]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [10]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [11]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

## Chain Creation

### LCEL (LangChain Expression Language)

Here, we use LCEL to combine various components into a single chain.

![lcel.png](./assets/02-langchain-expression-language.png)

```
chain = prompt | model | output_parser
```

The ```|``` symbol works similarly to the [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), linking different components and passing the output of one component as the input to the next.

In this chain, user input is passed to the prompt template, and the output from the prompt template is then forwarded to the model. By examining each component individually, you can understand what happens at each step.

In [12]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model

### Calling ```invoke()```

- Input values are provided in the form of a Python dictionary (key-value pairs).  
- When calling the ```invoke()``` function, these input values are passed as arguments.

In [13]:
# Set the topic in the `input` dictionary to 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [16]:
# Connect the `prompt` object and the `model` object using the pipe (`|`) operator.
# Use the `invoke` method to pass the `input`.
# This will return the message generated by the AI model.
chain.invoke(input)

AIMessage(content="Sure! The principles of learning in artificial intelligence (AI) models can be understood through a few key concepts. Here’s a simple breakdown:\n\n1. **Data**: AI models learn from data. Just like humans learn from experiences, AI systems learn from examples. The more quality data they have, the better they can learn.\n\n2. **Patterns**: AI looks for patterns in the data. For instance, if you show an AI many pictures of cats and dogs, it will try to find features that distinguish the two, like fur patterns or ear shapes.\n\n3. **Training**: This is the process where the AI model learns from the data. During training, the model adjusts its internal settings (called parameters) to improve its predictions or decisions based on the examples it sees.\n\n4. **Feedback**: After making predictions, the AI receives feedback on how well it did. This feedback helps the model adjust and improve. For example, if it incorrectly identifies a cat as a dog, it learns from that mista

Below is an example of outputting a streaming response:

In [17]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more high-quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can prepare.

2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want to teach an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. The model learns to identify patterns that distinguish cats from other objects.

3. **Feedback Loop**: AI models improve through feedback. After making predictions, they receive feedback on whether they were right or wrong. This feedback helps them adjust and make better predictions in the future, similar to how a coach helps an a

### Output Parser

An **Output Parser** is a tool designed to transform or process the responses from an AI model into a specific format. Since the model's output is typically provided as free-form text, an **Output Parser** is essential to convert it into a structured format or extract the required data.


In [19]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
)  # Directly returns the model's response as a string without modification.

An output parser is added to the chain.

In [20]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser

In [21]:
# Use the invoke method of the chain object to pass the input
chain.invoke(input)

'Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the fundamental ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:\n\n1. **Data-Driven Learning**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn patterns and make predictions. Think of it like a student learning from textbooks; the more information they have, the more they can understand.\n\n2. **Generalization**: This is the ability of an AI model to apply what it has learned from training data to new, unseen data. For example, if a model learns to recognize cats from pictures of cats, it should also be able to recognize a cat it has never seen before.\n\n3. **Feedback and Adjustment**: AI models often use feedback to improve. After making predictions, they can compare their results to the correct answers and adjust their internal parameters t

In [22]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more relevant and high-quality data you provide, the better the model can learn. Think of it like teaching a child; the more books and experiences they have, the more they learn.

2. **Patterns and Features**: AI looks for patterns in the data. It identifies important features (characteristics) that help it make decisions or predictions. For example, if you’re teaching an AI to recognize cats, it will learn features like fur, whiskers, and ears.

3. **Learning from Examples**: AI models learn by example. They are trained on a set of data (called the training set) where they see inputs and the correct outputs. Over time, they adjust their internal rules to improve their predictions.

4. **Feedback Loop**: After making predictions

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The ```model_name``` can also be adjusted for testing.

In [23]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

# Generate the prompt using the PromptTemplate
prompt = PromptTemplate.from_template(template)

# Initialize the ChatOpenAI model.
model = ChatOpenAI(model_name="gpt-4o-mini")

# Initialize the string output parser.
output_parser = StrOutputParser()

In [24]:
# Construct the chain.
chain = prompt | model | output_parser

In [ ]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))

- Dialogue in English:
**Student**: Hi, could you help me understand the differences between "restroom," "toilet," and "bathroom"? I always get confused.

**Teacher**: Of course! In American English, "restroom" is often used in public places, like restaurants or schools. It’s a polite way to refer to the facilities.

**Student**: Okay, so when would I use "toilet"?

**Teacher**: "Toilet" usually refers to the actual fixture or the room that contains the toilet. It's more commonly used in British English. 

**Student**: Got it! And what about "bathroom"?

**Teacher**: "Bathroom" generally refers to a room that has a toilet and a sink, and often a bathtub or shower as well. It’s usually used in homes.

**Student**: So, to summarize: "restroom" is for public places, "toilet" is the fixture, and "bathroom" is usually in a house?

**Teacher**: Exactly! You’ve got it!

---

- Explanation of the Dialogue:
In this conversation, a student seeks clarification on the terms "restroom," "toilet," a

In [28]:
# Execute the completed Chain to obtain a response
# Request for Streaming Output
answer = chain.stream({"question": "I want to go to a restaurant and order food."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- **Dialogue in English:**

**Customer:** Hi there! Can I see the menu, please?

**Waiter:** Of course! Here you go. Would you like anything to drink while you look?

**Customer:** Yes, I’d like a glass of water, please.

**Waiter:** Sure! I’ll get that for you right away. Have you decided on your meal?

**Customer:** Yes, I think I’ll have the grilled chicken salad.

**Waiter:** Great choice! Would you like any dressing on that?

**Customer:** Yes, can I have the vinaigrette, please?

**Waiter:** Absolutely! Is there anything else you’d like to add to your order?

**Customer:** No, that will be all for now. Thank you!

**Waiter:** You’re welcome! I’ll be back shortly with your water and salad.

---

- **Explanation of the Dialogue:**

In this conversation, the customer initiates the interaction by asking to see the menu, indicating they are ready to order. The waiter responds politely and offers a drink, showing good customer service. The customer orders a glass of water and then choo

In [32]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.
# Request for Streaming Output
answer = chain.stream({"question": "Ordering a quantum uranium pizza at a pub and being chased by the FBI"})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

**Dialogue in English:**

**Customer:** (nervously) Hi there! I’d like to order a quantum uranium pizza, please. 

**Waiter:** (raising an eyebrow) A what? I’ve never heard of that before. 

**Customer:** (glancing around) You know, the one with all the exotic toppings and a supercharged crust? 

**Waiter:** (confused) Uh, we just have regular pizzas here. Are you sure you don’t mean a supreme or something? 

**Customer:** (whispering) No, no! It’s very important I get that pizza. I’m in a bit of a hurry.

**Waiter:** (leaning closer) Why the rush? 

**Customer:** (suddenly looking terrified) Because the FBI is outside! 

**Waiter:** (wide-eyed) The FBI? What did you do? 

**Customer:** (frantically) I didn’t do anything! It’s all a misunderstanding! Just please, can you make that pizza quickly? 

**Waiter:** (nervously) Alright, alright! One quantum uranium pizza coming right up! 

**Customer:** (sighing in relief) Thank you! I’ll pay extra for it to go.

**Waiter:** (rushing to the k